In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import pandas as pd
import numpy as np
matplotlib.style.use('ggplot')
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.preprocessing import MinMaxScaler
import math
from torch.nn import MultiheadAttention

torch.manual_seed(33)

### Import Data

In [2]:
dna_meth = pd.read_csv('new_clean_data/consistent_dna_meth.csv')
dna_meth.drop(columns='Unnamed: 0', inplace=True)
dna_meth

,CELL_LINE_NAME,1:134999,1:135191;1:135218,1:135203;1:135208,1:713376;1:713388;1:713400;1:713448;1:713450;1:713454,1:713901;1:713921;1:714178;1:714182;1:714199;1:714254;1:714258;1:714261;1:714264;1:714277;1:714278;1:714293;1:714301;1:714491;1:714511;1:714566;1:714584,1:715390;1:715392;1:715405;1:715415,1:804993;1:804999;1:805282;1:805284;1:805290;1:805327;1:805338;1:805341;1:805352;1:805445;1:805450;1:805467;1:805468,1:805474;1:805477;1:805479,1:805484;1:805486,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,0.9821,1.00000,1.00000,0.41803,0.00000,0.23610,0.00000,0.10000,0.0000,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,0.8897,0.80360,0.99105,0.65648,0.00000,0.24032,0.00000,0.00000,0.0000,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,0.9222,0.87500,1.00000,0.49984,0.00897,0.95695,0.02262,0.12001,0.1765,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,1.0000,0.95915,1.00000,0.75370,0.00000,0.03540,0.07694,0.00000,0.0000,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,1.0000,1.00000,1.00000,0.70308,0.00000,0.16765,0.05868,0.16667,0.3000,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,wsudlcl2,0.7586,0.76473,1.00000,0.70492,0.00000,0.72915,0.00000,0.00000,0.0000,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
539,yapc,0.9595,0.98215,1.00000,0.96413,0.00000,0.86138,0.00357,0.00000,0.0000,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
540,yh13,0.9211,1.00000,0.97220,0.59999,0.00000,0.94020,0.01222,0.00000,0.0000,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
541,ykg1,0.8617,0.92310,0.93395,0.82007,0.00000,0.99590,0.00000,0.00000,0.0000,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [3]:
gene_exp = pd.read_csv('new_clean_data/consistent_gene_exp.csv')
gene_exp.drop(columns='Unnamed: 0', inplace=True)
gene_exp

,CELL_LINE_NAME,ENSG00000000003.10,ENSG00000000005.5,ENSG00000000419.8,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000938.8,ENSG00000000971.11,ENSG00000001036.9,ENSG00000001084.6,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,5.28,0.0,73.38,9.76,24.51,0.01,0.08,54.86,118.50,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,7.01,0.0,108.99,16.76,13.32,0.00,0.23,170.91,93.00,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,23.09,0.0,99.28,2.73,9.27,0.02,0.45,53.29,8.36,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,57.94,0.0,98.95,3.11,31.61,0.14,1.98,71.97,9.76,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,32.02,0.0,125.99,2.69,10.45,0.00,12.89,82.50,24.13,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,wsudlcl2,0.74,0.0,41.71,5.60,20.21,30.57,0.46,11.78,8.32,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
539,yapc,50.29,0.0,103.70,7.76,12.84,0.17,13.58,71.13,29.67,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
540,yh13,28.92,0.0,64.09,4.94,13.35,0.22,139.44,118.19,13.53,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
541,ykg1,61.08,0.0,109.13,5.91,17.40,0.13,53.25,92.96,23.09,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [4]:
# get columns
cell_line_cols = ['CELL_LINE_NAME']
drug_resp_cols = ['Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']
dna_meth_cols = dna_meth.drop(columns=['CELL_LINE_NAME','Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']).columns
gene_exp_cols = gene_exp.drop(columns=['CELL_LINE_NAME','Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']).columns

In [5]:
# drug response labels to tensor
drug_resp_dna = np.stack([dna_meth[col].values for col in drug_resp_cols], 1)
drug_resp_dna = torch.tensor(drug_resp_dna, dtype=torch.float)
drug_resp_gene = np.stack([gene_exp[col].values for col in drug_resp_cols], 1)
drug_resp_gene = torch.tensor(drug_resp_gene, dtype=torch.float)
drug_resp = drug_resp_dna

In [6]:
# dna_meth values to tensor
dna_meth_values = np.stack([dna_meth[col].values for col in dna_meth_cols], 1)
dna_meth_values = torch.tensor(dna_meth_values, dtype=torch.float)

In [7]:
# gene_exp values to tensor and scale from 0 to 1
gene_exp_values = np.stack([gene_exp[col].values for col in gene_exp_cols], 1)
scaler = MinMaxScaler()
gene_exp_values_t = gene_exp_values.T
scaled_gene_exp_values_t = scaler.fit_transform(gene_exp_values_t)
scaled_gene_exp_values = scaled_gene_exp_values_t.T
scaled_gene_exp_values = torch.tensor(scaled_gene_exp_values, dtype=torch.float)

### Define Dataloaders

In [8]:
batch_size = 32

full_dataset = torch.utils.data.TensorDataset(dna_meth_values, scaled_gene_exp_values, drug_resp)
full_loader = DataLoader(full_dataset, shuffle=False, batch_size=batch_size)

### VAE model with Regressor

In [9]:
class VAE_with_regressor(nn.Module):
    def __init__(self, input_size_dna, input_size_gene, level_2, level_3, latent_dim, shared_size, independent_size):
        super(VAE_with_regressor, self).__init__()

        self.input_size_dna = input_size_dna
        self.input_size_gene = input_size_gene
        self.level_2 = level_2
        self.level_3 = level_3
        self.latent_dim = latent_dim
        self.shared_size = shared_size
        self.independent_size = independent_size

        # Encoder DNA
        self.enc_fc1_dna = nn.Sequential(
                        nn.Linear(input_size_dna, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.enc_fc2_dna = nn.Sequential(
                        nn.Linear(level_2, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.enc_fc3_dna_mean = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        self.enc_fc3_dna_log_var = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        # Decoder DNA
        self.dec_fc3_dna = nn.Sequential(
                        nn.Linear(latent_dim, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.dec_fc2_dna = nn.Sequential(
                        nn.Linear(level_3, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.dec_fc1_dna = nn.Sequential(
                    nn.Linear(level_2, input_size_dna),
                    nn.BatchNorm1d(input_size_dna),
                    nn.Sigmoid())
        
        # Encoder Gene
        self.enc_fc1_gene = nn.Sequential(
                        nn.Linear(input_size_gene, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.enc_fc2_gene = nn.Sequential(
                        nn.Linear(level_2, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.enc_fc3_gene_mean = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        self.enc_fc3_gene_log_var = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        # Decoder Gene
        self.dec_fc3_gene = nn.Sequential(
                        nn.Linear(latent_dim, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.dec_fc2_gene = nn.Sequential(
                        nn.Linear(level_3, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.dec_fc1_gene = nn.Sequential(
                    nn.Linear(level_2, input_size_gene),
                    nn.BatchNorm1d(input_size_gene),
                    nn.Sigmoid())
        
        # attention layer
        self.attention = MultiheadAttention(embed_dim=(independent_size * 2 + shared_size), num_heads=int((independent_size * 2 + shared_size)))
        
        # Regression fc layers
        self.r_fc1 = nn.Sequential(
                    nn.Linear(independent_size * 2 + shared_size, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU())
        self.r_fc2 = nn.Sequential(
                    nn.Linear(64, 32),
                    nn.BatchNorm1d(32),
                    nn.ReLU())
        
        self.r_fc3 = nn.Sequential(
                    nn.Linear(32, 10),
                    nn.BatchNorm1d(10))
        
    def encode(self, x):
        dna_l2_layer = self.enc_fc1_dna(x[0])
        dna_l3_layer = self.enc_fc2_dna(dna_l2_layer)
        mu_dna = self.enc_fc3_dna_mean(dna_l3_layer)
        logvar_dna = self.enc_fc3_dna_log_var(dna_l3_layer)

        gene_l2_layer = self.enc_fc1_gene(x[1])
        gene_l3_layer = self.enc_fc2_gene(gene_l2_layer)
        mu_gene = self.enc_fc3_gene_mean(gene_l3_layer)
        logvar_gene = self.enc_fc3_gene_log_var(gene_l3_layer)

        u_dna, s_dna = torch.split(mu_dna, [self.independent_size, self.shared_size], -1)
        u_gene, s_gene = torch.split(mu_gene, [self.independent_size, self.shared_size], -1)
        shared_mean = torch.mean(torch.stack([s_dna, s_gene]), dim=0)
        for_pred = torch.cat((u_dna, shared_mean, u_gene), dim=1)

        mu_for_pred = for_pred.unsqueeze(0).transpose(0, 1)
        attended_mean, _ = self.attention(mu_for_pred,mu_for_pred,mu_for_pred)
        attended_mean = attended_mean.transpose(0, 1).squeeze(0)
        # attended_mean = for_pred

        return mu_dna, logvar_dna, mu_gene, logvar_gene, attended_mean, u_dna, s_dna, u_gene, s_gene
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z
    
    def decode_dna(self, z):
        l3_dna_layer = self.dec_fc3_dna(z)
        l2_dna_layer = self.dec_fc2_gene(l3_dna_layer)
        recon_dna = self.dec_fc1_dna(l2_dna_layer)
        return recon_dna
    
    def decode_gene(self,z):
        l3_gene_layer = self.dec_fc3_gene(z)
        l2_gene_layer = self.dec_fc2_gene(l3_gene_layer)
        recon_gene = self. dec_fc1_gene(l2_gene_layer)
        return recon_gene
    
    def regressor(self, mean):
        level_1_layer = self.r_fc1(mean)
        level_2_layer = self.r_fc2(level_1_layer)
        output_layer = self.r_fc3(level_2_layer)
        return output_layer
    
    def forward(self, x):
        mu_dna, logvar_dna, mu_gene, logvar_gene, attended_mean, u_dna, s_dna, u_gene, s_gene = self.encode(x)
        z_dna = self.reparameterize(mu_dna, logvar_dna)
        z_gene = self.reparameterize(mu_gene, logvar_gene)
        recon_dna = self.decode_dna(z_dna)
        recon_gene = self.decode_gene(z_gene)

        
        y_pred = self.regressor(attended_mean)

        return recon_dna, recon_gene, mu_dna, logvar_dna, mu_gene, logvar_gene, u_dna, s_dna, u_gene, s_gene, y_pred

### Define loss functions

In [10]:
def vae_loss_function(x_hat, x, mean, log_var): # recon loss and kld loss
        bce = torch.nn.functional.binary_cross_entropy(x_hat, x, reduction = 'sum')
        kld = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
        loss = kld + bce
        return loss, kld, bce

In [11]:
def rmse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = torch.sqrt(loss_fn(y_pred, y))
    return loss

In [12]:
def mse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = loss_fn(y_pred, y)
    return loss

In [13]:
def mse_loss_function_mean(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='mean')
    loss = loss_fn(y_pred, y)
    return loss

In [14]:
class IndependenceLoss(nn.Module):
    def __init__(self):
        super(IndependenceLoss, self).__init__()

    def forward(self, batch1, batch2):
        # Compute the covariance matrix for each pair of rows
        cov_matrix = torch.matmul(batch1.unsqueeze(2), batch2.unsqueeze(1))

        # Compute the Frobenius norm of each covariance matrix
        loss = torch.norm(cov_matrix, dim=(1, 2), p='fro')

        loss_mean = loss.mean()

        return loss_mean

In [15]:
independence_loss = IndependenceLoss()

### Validation Functions

In [16]:
def r_squared(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    SS_res = torch.sum(torch.square(y_true - y_pred))
    SS_tot = torch.sum(torch.square(y_true - y_true_mean))
    r_squared = 1 - SS_res / (SS_tot + torch.finfo(torch.float32).eps)
    return r_squared

### Training Parameters

In [17]:
epochs = 50

lr = 0.001
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

k = 10
splits = KFold(n_splits=k, shuffle=False)

In [18]:
# Initialize the model
input_size_dna = 81037
input_size_gene = 57820 #dimension of gene expressions
level_2 = 2048
level_3 = 1024
latent_dim = 128 # target latent size

shared_size = 10
independent_size = 118

model = VAE_with_regressor(input_size_dna, input_size_gene, level_2, level_3, latent_dim, shared_size, independent_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

### Train Function

In [19]:
def train (model, dataloader, epoch, optimizer, w_recon_loss, w_kl_loss, w_reg_loss, w_shared_loss, w_ind_loss):
    model.train()
    train_recon = 0
    train_kl  = 0
    train_reg = 0
    train_shared = 0
    train_ind = 0
    batch_num = 0

    for batch_idx, batch in enumerate(dataloader):
        batch_num = batch_num + 1
        # get values
        dna_meth_values, gene_exp_values, drug_resp = batch
        dna_meth_values = dna_meth_values.to(device)
        gene_exp_values = gene_exp_values.to(device)
        drug_resp = drug_resp.to(device)

        dna_plus_gene = [dna_meth_values, gene_exp_values]
        dna_plus_gene = dna_plus_gene

        
        # clear gradients
        optimizer.zero_grad()
        # apply model
        recon_dna, recon_gene, mu_dna, logvar_dna, mu_gene, logvar_gene, u_dna, s_dna, u_gene, s_gene, y_pred = model(dna_plus_gene)

        # losses
        dna_vae_loss, dna_kld, dna_bce = vae_loss_function(recon_dna, dna_meth_values, mu_dna, logvar_dna)
        gene_vae_loss, gene_kld, gene_bce = vae_loss_function(recon_gene, gene_exp_values, mu_gene, logvar_gene)

        shared_loss = mse_loss_function_mean(s_dna, s_gene)
        ind_loss = independence_loss(u_dna, u_gene)

        reg_loss = mse_loss_function_mean(y_pred, drug_resp)

        recon_loss = 0.5 * (dna_bce + gene_bce)
        kld_loss = 0.5 * (dna_kld + gene_kld)

        loss = w_recon_loss * recon_loss + w_kl_loss * kld_loss + w_reg_loss * reg_loss + w_shared_loss * shared_loss + w_ind_loss * ind_loss


        # backward
        loss.backward()

        with torch.no_grad():
            train_recon += recon_loss.item()
            train_kl += kld_loss.item()
            train_reg += reg_loss.item()
            train_shared += shared_loss.item()
            train_ind += ind_loss.item()

        optimizer.step()
    
    train_recon_ave = train_recon/ len(dataloader.sampler)
    train_kl_ave = train_kl/ len(dataloader.sampler)
    train_reg_ave = train_reg/ batch_num
    train_shared_ave = train_shared/ batch_num
    train_ind_ave = train_ind/batch_num

    print('=====> Epoch {} \n' 
          'Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average Regression Loss: {:.3f} \n'
          'Average Shared Loss: {:.3f} \n'
          'Average Independence Loss: {:.3f} \n'.format(epoch, train_recon_ave, train_kl_ave, train_reg_ave, train_shared_ave, train_ind_ave))
    return train_recon_ave, train_kl_ave, train_reg_ave, train_shared_ave, train_ind_ave
        

### Validation Function

In [20]:
def test(dataloader, model):
    model.eval()

    test_recon_dna = 0
    test_recon_gene = 0
    test_kl_dna = 0
    test_kl_gene = 0

    test_reg = 0
    test_rmse = 0
    test_shared = 0
    test_shared_rmse = 0
    test_r2 = 0
    test_ind_loss = 0
    

    batch_num = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            batch_num = batch_num + 1
            
            # get data values
            dna_meth_values, gene_exp_values, drug_resp = batch
            dna_meth_values = dna_meth_values.to(device)
            gene_exp_values = gene_exp_values.to(device)
            drug_resp = drug_resp.to(device)

            dna_plus_gene = [dna_meth_values, gene_exp_values]

            recon_dna, recon_gene, mu_dna, logvar_dna, mu_gene, logvar_gene, u_dna, s_dna, u_gene, s_gene, y_pred = model(dna_plus_gene)

            # losses
            dna_vae_loss, dna_kld, dna_bce = vae_loss_function(recon_dna, dna_meth_values, mu_dna, logvar_dna)
            gene_vae_loss, gene_kld, gene_bce = vae_loss_function(recon_gene, gene_exp_values, mu_gene, logvar_gene)

            shared_loss = mse_loss_function(s_dna, s_gene)
            ind_loss = independence_loss(u_dna, u_gene)

            reg_loss = mse_loss_function(y_pred, drug_resp)
            r2_value = r_squared(drug_resp, y_pred)

            test_recon_dna += dna_bce.item()
            test_recon_gene += gene_bce.item()
            test_kl_dna += dna_kld.item()
            test_kl_gene += gene_kld.item()

            test_reg += reg_loss.item()
            test_shared += shared_loss.item()
            test_r2 += r2_value.item()
            test_ind_loss += ind_loss.item()

        # print loss
        test_recon_dna_ave = test_recon_dna/len(dataloader.sampler)
        test_recon_gene_ave = test_recon_gene/len(dataloader.sampler)
        test_kl_dna_ave = test_kl_dna/ len(dataloader.sampler)
        test_kl_gene_ave = test_kl_gene/ len(dataloader.sampler)

        test_reg_ave = test_reg/ (len(dataloader.sampler) * 10)
        test_rmse_ave = math.sqrt(test_reg_ave)
        test_shared_ave = test_shared/ (len(dataloader.sampler) * 10)
        test_shared_rmse_ave = math.sqrt(test_shared_ave)
        test_r2_ave = test_r2/ batch_num
        test_ind_ave = test_ind_loss/ batch_num

        print('Average DNA Recon Loss: {:.3f} \n'
              'Average Gene Recon Loss: {:.3f} \n'
          'Average DNA KL Loss: {:.3f} \n'
          'Average Gene KL Loss: {:.3f} \n'
          'Average Regressor Loss: {:.3f} \n'
          'Average RMSE Loss: {:.3f} \n'
          'Average Shared MSE Loss: {:.3f} \n'
          'Average Shared RMSE Loss: {:.3f} \n'
          'Average R2: {:.3f} \n'
          'Average Indepence Loss: {:.3f} \n'.format(test_recon_dna_ave, test_recon_gene_ave, test_kl_dna_ave, test_kl_gene_ave, test_reg_ave, test_rmse_ave, test_shared_ave, test_shared_rmse_ave, test_r2_ave, test_ind_ave))
        
        return test_recon_dna_ave, test_recon_gene_ave, test_kl_dna_ave, test_kl_gene_ave, test_reg_ave, test_rmse_ave, test_shared_ave, test_shared_rmse_ave, test_r2_ave, test_ind_ave

### Execute Training and K fold Cross Validation

In [21]:
history_train = {'Train Recon Loss': [], 'Train KL Loss': [], 'Train MSE Loss': [], 'Train Shared Loss': [], 'Train Independence Loss': []}
history_test = {'Test DNA Recon Loss': [],'Test Gene Recon Loss': [], 'Test KL DNA Loss': [], 'Test KL Gene Loss': [], 'Test MSE Loss': [], 'Test RMSE Loss': [], 'Test Shared Loss': [], 'Test Shared RMSE Loss': [],  'Test R2': [], 'Test Independence Loss': []}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(full_dataset)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=test_sampler)

    print("Start Training (Unsupervised Phase)")
    for epoch in range(1, epochs + 1):
        train_recon_ave, train_kl_ave, train_reg_ave, train_shared_ave, train_ind_ave = train(model, train_loader, epoch, optimizer, w_recon_loss= 1, w_kl_loss= 1, w_reg_loss= 0, w_shared_loss= 0, w_ind_loss= 0)
        history_train['Train Recon Loss'].append(train_recon_ave)
        history_train['Train KL Loss'].append(train_kl_ave)
        history_train['Train MSE Loss'].append(train_reg_ave)
        history_train['Train Shared Loss'].append(train_shared_ave)
        history_train['Train Independence Loss'].append(train_ind_ave)
    
    print("Start Training (Supervised Phase)")
    for epoch in range(1, epochs + 1):
        train_recon_ave, train_kl_ave, train_reg_ave, train_shared_ave, train_ind_ave = train(model, train_loader, epoch, optimizer, w_recon_loss= 0, w_kl_loss= 0, w_reg_loss= 1, w_shared_loss= 1, w_ind_loss=1)
        history_train['Train Recon Loss'].append(train_recon_ave)
        history_train['Train KL Loss'].append(train_kl_ave)
        history_train['Train MSE Loss'].append(train_reg_ave)
        history_train['Train Shared Loss'].append(train_shared_ave)
        history_train['Train Independence Loss'].append(train_ind_ave)

    test_recon_dna_ave, test_recon_gene_ave, test_kl_dna_ave, test_kl_gene_ave, test_reg_ave, test_rmse_ave, test_shared_ave, test_shared_rmse_ave, test_r2_ave, test_ind_ave = test(test_loader, model)
    history_test['Test DNA Recon Loss'].append(test_recon_dna_ave)
    history_test['Test Gene Recon Loss'].append(test_recon_gene_ave)
    history_test['Test KL DNA Loss'].append(test_kl_dna_ave)
    history_test['Test KL Gene Loss'].append(test_kl_gene_ave)
    history_test['Test MSE Loss'].append(test_reg_ave)
    history_test['Test RMSE Loss'].append(test_rmse_ave)
    history_test['Test Shared Loss'].append(test_shared_ave)
    history_test['Test Shared RMSE Loss'].append(test_shared_rmse_ave)
    history_test['Test R2'].append(test_r2_ave)
    history_test['Test Independence Loss'].append(test_ind_ave)
    

Fold 1
Start Training (Unsupervised Phase)
=====> Epoch 1 
Average Recon Loss: 51375.049 
Average KL Loss: 125.651 
Average Regression Loss: 13.661 
Average Shared Loss: 2.104 
Average Independence Loss: 85.986 

=====> Epoch 2 
Average Recon Loss: 49439.650 
Average KL Loss: 117.515 
Average Regression Loss: 13.620 
Average Shared Loss: 2.045 
Average Independence Loss: 84.044 

=====> Epoch 3 
Average Recon Loss: 48644.972 
Average KL Loss: 113.918 
Average Regression Loss: 13.531 
Average Shared Loss: 2.122 
Average Independence Loss: 81.823 

=====> Epoch 4 
Average Recon Loss: 47987.071 
Average KL Loss: 108.671 
Average Regression Loss: 13.484 
Average Shared Loss: 2.045 
Average Independence Loss: 86.728 

=====> Epoch 5 
Average Recon Loss: 47458.685 
Average KL Loss: 105.755 
Average Regression Loss: 13.642 
Average Shared Loss: 2.095 
Average Independence Loss: 86.389 

=====> Epoch 6 
Average Recon Loss: 46956.104 
Average KL Loss: 108.448 
Average Regression Loss: 13.716 
A

### Performance of model through 5 Fold CV

In [22]:
cv_dna_recon_loss = np.mean(history_test['Test DNA Recon Loss'])
cv_gene_recon_loss = np.mean(history_test['Test Gene Recon Loss'])
cv_dna_kl_loss = np.mean(history_test['Test KL DNA Loss'])
cv_gene_kl_loss = np.mean(history_test['Test KL Gene Loss'])
cv_mse_loss = np.mean(history_test['Test MSE Loss'])
cv_rmse_loss = np.mean(history_test['Test RMSE Loss'])
cv_shared_loss = np.mean(history_test['Test Shared Loss'])
cv_shared_rmse_loss = np.mean(history_test['Test Shared RMSE Loss'])
cv_r2_loss = np.mean(history_test['Test R2'])
cv_ind_loss = np.mean(history_test['Test Independence Loss'])

print('Performance of {} fold cross validation using common and unique factors:'.format(k))
print("\nAverage Testing DNA Recon Loss: {:.4f} \nAverage Testing Gene Recon Loss: {:.4f} \n Average Testing DNA KL Loss: {:.4f} \n Average Testing Gene KL Loss: {:.4f} \n Average Testing MSE Loss: {:.3f} \n Average Testing RMSE Loss: {:.3f} \n Average Testing Shared Loss: {:.3f} \n Average Testing Shared RMSE Loss: {:.3f} \n Average Testing R2 Loss: {:.3f} \n Average Testing Independence Loss: {:.3f} \n".format(cv_dna_recon_loss, cv_gene_recon_loss, cv_dna_kl_loss, cv_gene_kl_loss, cv_mse_loss, cv_rmse_loss, cv_shared_loss,cv_shared_rmse_loss, cv_r2_loss, cv_ind_loss))

Performance of 10 fold cross validation using common and unique factors:

Average Testing DNA Recon Loss: 30002.3058 
Average Testing Gene Recon Loss: 5090.4284 
 Average Testing DNA KL Loss: 172.9617 
 Average Testing Gene KL Loss: 9.0482 
 Average Testing MSE Loss: 2.614 
 Average Testing RMSE Loss: 1.526 
 Average Testing Shared Loss: 1.726 
 Average Testing Shared RMSE Loss: 1.304 
 Average Testing R2 Loss: 0.702 
 Average Testing Independence Loss: 6.749 



In [23]:
history_test['Test R2']

[0.17159724235534668,
 0.4582054018974304,
 0.5908139050006866,
 0.7051718235015869,
 0.8294875621795654,
 0.8394534885883331,
 0.8368826806545258,
 0.8399815261363983,
 0.8721919655799866,
 0.8772014677524567]